In [28]:
import pandas as pd
import kaggle
import re

In [29]:
kaggle.api.authenticate()

In [30]:
# https://www.kaggle.com/oscarfry/tvtime-shows?select=tvtimeshows.csv
kaggle.api.dataset_download_files('oscarfry/tvtime-shows', path='./data/tvtimeshows', unzip = True)

df_tvtime = pd.read_csv('data/tvtimeshows/tvtimeshows.csv')
df_tvtime.head()

,id,name,followers,nb_rates,runtime,number_of_seasons,network,mean_rate,poster_image,seasons
0,288128,X-Ray & Vav,230,10,9,2,Rooster Teeth,4.20,https://dg31sz3gwrwan.cloudfront.net/poster/28...,"[{'number': 1, 'nb_episodes': 4}, {'number': 2..."
1,313803,Sister's Slam Dunk,1298,100,80,2,KBS TV2,4.25,https://dg31sz3gwrwan.cloudfront.net/poster/31...,"[{'number': 1, 'nb_episodes': 33}, {'number': ..."
2,325462,Reverse,270,15,60,1,Tokyo Broadcasting System,4.63,https://dg31sz3gwrwan.cloudfront.net/poster/32...,"[{'number': 1, 'nb_episodes': 10}]"
3,325198,Frame Arms Girl,900,24,25,1,Tokyo MX,3.54,https://dg31sz3gwrwan.cloudfront.net/poster/32...,"[{'number': 1, 'nb_episodes': 12}]"
4,349743,Mr Inbetween,7123,118,25,2,FX,4.42,https://dg31sz3gwrwan.cloudfront.net/poster/34...,"[{'number': 1, 'nb_episodes': 6}, {'number': 2..."


In [31]:
# https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney 
kaggle.api.dataset_download_files('ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney', path='./data/streamings', unzip = True)

df_streamings = pd.read_csv('data/streamings/tv_shows.csv')
df_streamings.head()

,Unnamed: 0,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,type
0,0,Breaking Bad,2008,18+,9.5,96%,1,0,0,0,1
1,1,Stranger Things,2016,16+,8.8,93%,1,0,0,0,1
2,2,Money Heist,2017,18+,8.4,91%,1,0,0,0,1
3,3,Sherlock,2010,16+,9.1,78%,1,0,0,0,1
4,4,Better Call Saul,2015,18+,8.7,97%,1,0,0,0,1


In [32]:
# https://www.kaggle.com/unanimad/emmy-awards
kaggle.api.dataset_download_files('unanimad/emmy-awards', path='./data/awards', unzip = True)

df_award_emmy = pd.read_csv('data/awards/the_emmy_awards.csv')
df_award_emmy.head()

,id,year,category,nominee,staff,company,producer,win
0,1,2019,Outstanding Character Voice-Over Performance,The Simpsons,"Hank Azaria, as Moe, Carl, Duffman, Kirk",FOX,Gracie Films in association with 20th Century ...,False
1,2,2019,Outstanding Character Voice-Over Performance,Family Guy,"Alex Borstein, as Lois Griffin, Tricia Takanawa",FOX,20th Century Fox Television,False
2,3,2019,Outstanding Character Voice-Over Performance,When You Wish Upon A Pickle: A Sesame Street S...,"Eric Jacobson, as Bert, Grover, Oscar",HBO,Sesame Street Workshop,False
3,4,2019,Outstanding Character Voice-Over Performance,F Is For Family,"Kevin Michael Richardson, as Rosie",Netflix,Wild West Television in association with Gaumo...,False
4,5,2019,Outstanding Production Design For A Narrative ...,Escape At Dannemora,"Mark Ricker, Production Designer; James Truesd...",Showtime,"Red Hour, Busyhands, The White Mountain Compan...",False


In [33]:
# https://www.kaggle.com/unanimad/golden-globe-awards
kaggle.api.dataset_download_files('unanimad/golden-globe-awards', path='./data/awards' , unzip = True)

df_award_goldenglobe = pd.read_csv('data/awards/golden_globe_awards.csv')
df_award_goldenglobe.head()

,year_film,year_award,ceremony,category,nominee,film,win
0,1943,1944,1,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True
1,1943,1944,1,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True
2,1943,1944,1,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True
3,1943,1944,1,Picture,The Song Of Bernadette,NaN,True
4,1943,1944,1,Actress In A Leading Role,Jennifer Jones,The Song Of Bernadette,True


In [34]:
df_tvstatus = pd.read_csv('status/metacritic.csv')
df_tvstatus.head()

,name,season,status,tv_network,ano
0,20/20,SEASON 36,Renewed 5/13,ABC,2013
1,America's Funniest Home Videos,SEASON 24,Renewed 5/9,ABC,2013
2,The Assets,NaN,Canceled 1/10 after 2 eps aired,ABC,2013
3,The Astronaut Wives Club,Starts tbd,Delayed to 2015 for creative reasons,ABC,2013
4,The Bachelor,SEASON 18,Renewed 5/9,ABC,2013
